# Overview
Use both records and stream API endpoints to write / read from
a Gretel Project in near real-time. This notebook will create
a temporary project and send fake records and continuously
consume the labeled records.



In [1]:
!pip install -Uqq gretel-client

In [ ]:
# Get an API key to connect to the Gretel APIs 
# from https://console.gretel.cloud

from gretel_client import get_cloud_client

client = get_cloud_client(prefix="api", api_key="prompt")

In [ ]:
import sys
import threading
import time

from faker import Faker

from gretel_client.client import temporary_project, Client
from gretel_client.projects import Project


def subscriber(project: Project):
    # By default, ``iter_records()`` will block waiting for
    # new labeled records to be ready for consumption. It will
    # fetch in batches, so if there are already records in the
    # stream, we will immediately yield the most recent 200 records.
    for rec in project.iter_records():
        yield rec


def publish(project: Project, event: threading.Event):
    fake = Faker()
    while not event.is_set():
        rec = {
            "name": fake.name(),
            "phone": fake.phone_number()
        }
        project.send(rec)
        time.sleep(1)


def start(client: Client):
    with temporary_project(client) as project:
        publish_event = threading.Event()
        publish_thread = threading.Thread(target=publish, args=(project, publish_event))
        publish_thread.start()
        try:
            for rec in subscriber(project):
                print(rec)
        except KeyboardInterrupt:
            print("Shutting down...")
        finally:
            publish_event.set()
            publish_thread.join()


start(client)